In [1]:
import numpy as np
import librosa
import glob
import pickle
from tqdm import tqdm
from sklearn.preprocessing import normalize

from MusicVectorizer import MusicVectorizer

SAMPLE_RATE = 16000

data_folders = ["90s_hiphop", "90s_rock", "2010s_pop", "classical", "country"]


trajectories = {}
for folder in data_folders:
    trajectories[folder] = np.load("data/" + folder + "_trajectories.npy", allow_pickle=True).item()

/Users/bensmacbook/.pyenv/versions/3.10.6/envs/flask-448/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
trajectories.keys()

dict_keys(['90s_hiphop', '90s_rock', '2010s_pop', 'classical', 'country'])

In [3]:
def split_song( song : np.ndarray, num_songlets : int, norm : bool) -> np.ndarray:
    """
    Splits a single song that is (num_timesteps, 768) into (num_songlets, 768).
    """

    songlet_size = song.shape[0] // num_songlets
    songlets = []

    for i in range(num_songlets):
        if i == num_songlets - 1:
            songlets.append(song[i*songlet_size:])
        else:
            songlets.append(song[i*songlet_size:(i+1)*songlet_size])

    if norm:
        return normalize([np.sum(s, axis=0) for s in songlets])
    else:
        return np.stack([np.sum(s, axis=0) for s in songlets])

In [4]:
for genre in trajectories.keys():
    for song in trajectories[genre].keys():
        trajectories[genre][song] = trajectories[genre][song][:,12,:]
        trajectories[genre][song] = trajectories[genre][song].reshape(trajectories[genre][song].shape[0], -1)
        trajectories[genre][song] = split_song(trajectories[genre][song], 10, True)

In [5]:
for genre in trajectories.keys():
    for song in trajectories[genre].keys():
        print(trajectories[genre][song].shape)

(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)
(10, 768)


# ANNOY

In [6]:
from annoy import AnnoyIndex
mapping = {}

ModuleNotFoundError: No module named 'annoy'

In [94]:
sim = AnnoyIndex(768, 'angular')
idx = 0
for genre in trajectories.keys():
    for song in trajectories[genre].keys():
        for i in range(trajectories[genre][song].shape[0]):
            sim.add_item(idx, trajectories[genre][song][0])
            mapping[idx] = (genre, song)
            idx += 1

In [95]:
idx

19218

In [19]:
# add known songs to annoy index and find similarities
hotel_room_service = np.load("temp/JGwWNGJdvx8.npy")
hotel_room_service = hotel_room_service[:,12,:]
hotel_room_service = hotel_room_service.reshape(hotel_room_service.shape[0], -1)
shape_of_you = split_song(hotel_room_service, 10, True)
print(shape_of_you.shape)

(10, 768)


In [97]:
sim.build(1000)

True

# FLANN

In [1]:
from pyflann import *

ModuleNotFoundError: No module named 'pyflann'

In [113]:
dataset = []
for song in trajectories[genre].keys():
    for i in range(trajectories[genre][song].shape[0]):
            dataset.append(trajectories[genre][song][0])
dataset = np.array(dataset)

In [115]:
hotel_room_service.shape

(104, 768)

In [114]:
flann = FLANN()

In [129]:
results, dists = flann.nn(dataset, hotel_room_service, 10, algorithm="kmeans", branching=32, iterations=7, checks=16)

In [130]:
results = np.array(results)
print(results.shape)

(104, 10)


# FLANN Evaluation

In [131]:
flann_counts = {}
for i in range(len(results)):
    for result in results[i]:
        if result in flann_counts:
            flann_counts[result] += 1
        else:
            flann_counts[result] = 1

In [133]:
# flann most common
flann_most_common = sorted(flann_counts.items(), key=lambda x: x[1], reverse=True)
print(flann_most_common)

[(3343, 32), (3344, 32), (3345, 32), (3346, 32), (3347, 32), (3348, 32), (3349, 32), (3350, 32), (3351, 32), (3352, 32), (0, 22), (1, 22), (2, 22), (3, 22), (4, 22), (5, 22), (6, 22), (7, 22), (8, 22), (9, 22), (1717, 15), (1718, 15), (1719, 15), (1720, 15), (1721, 15), (1722, 15), (1723, 15), (1724, 15), (1725, 15), (1726, 15), (610, 9), (611, 9), (612, 9), (613, 9), (614, 9), (615, 9), (616, 9), (617, 9), (618, 9), (619, 9), (1025, 7), (1026, 7), (1027, 7), (1028, 7), (1029, 7), (1030, 7), (1031, 7), (1032, 7), (1033, 7), (1034, 7), (3188, 4), (3189, 4), (3190, 4), (3191, 4), (3192, 4), (3193, 4), (3194, 4), (3195, 4), (3196, 4), (3197, 4), (3569, 3), (3570, 3), (3571, 3), (3572, 3), (3573, 3), (3574, 3), (3575, 3), (3576, 3), (3577, 3), (3578, 3), (3140, 2), (3141, 2), (3142, 2), (3143, 2), (3144, 2), (3145, 2), (3146, 2), (3147, 2), (3148, 2), (3149, 2), (41, 2), (42, 2), (43, 2), (44, 2), (45, 2), (46, 2), (47, 2), (48, 2), (49, 2), (50, 2), (436, 2), (437, 2), (438, 2), (439, 2),

In [136]:
flann_unique = []
for item in flann_most_common:
    if mapping[item[0]][1] not in flann_unique:
        flann_unique.append(mapping[item[0]][1])

In [138]:
for item in flann_unique[:10]:
    print(item)

data/90s_hiphop/Craig Mack - Flava In Ya Ear (Official Music Video).mp3
data/90s_hiphop/Missy Elliott - The Rain (Supa Dupa Fly) [Official Music Video].mp3
data/90s_hiphop/Bone Thugs N Harmony - 1st of tha Month.mp3
data/90s_hiphop/MC Lyte & Missy Elliott - Cold Rock a Party.mp3
data/90s_hiphop/MC Lyte - Ruffneck (Official Video) [Explicit].mp3
data/90s_hiphop/Das EFX - They Want EFX.mp3
data/90s_hiphop/Puff Daddy - It's All About The Benjamins (Remix) (Official Music Video) [HD].mp3
data/90s_hiphop/Outkast - ATliens (Official HD Video).mp3
data/90s_hiphop/Bone Thugs N Harmony - Crossroads.mp3
data/90s_hiphop/Dr. Dre - Still D.R.E. ft. Snoop Dogg.mp3


# ANNOY Evaluation Code

In [134]:
counts = {}
for i in range(hotel_room_service.shape[0]):
    results = sim.get_nns_by_vector(hotel_room_service[i], 10, search_k=-1, include_distances=False)
    for result in results:
        if result in counts:
            counts[result] += 1
        else:
            counts[result] = 1

In [135]:
# get 10 most common counts
most_common = sorted(counts.items(), key=lambda x: x[1], reverse=True)
print(most_common)

[(946, 20), (947, 20), (948, 20), (949, 20), (950, 20), (951, 20), (952, 20), (953, 20), (954, 20), (955, 20), (1565, 19), (1566, 19), (1567, 19), (1568, 19), (1569, 19), (1570, 19), (1571, 19), (1572, 19), (1573, 19), (1574, 19), (3760, 18), (3761, 18), (3762, 18), (3763, 18), (3764, 18), (3765, 18), (3766, 18), (3767, 18), (3768, 18), (3769, 18), (1772, 14), (1773, 14), (1774, 14), (1775, 14), (1776, 14), (1777, 14), (1778, 14), (1779, 14), (1780, 14), (1781, 14), (16224, 4), (16225, 4), (16226, 4), (16227, 4), (16228, 4), (16229, 4), (16230, 4), (16231, 4), (16232, 4), (16233, 4), (17331, 4), (17332, 4), (17333, 4), (17334, 4), (17335, 4), (17336, 4), (17337, 4), (17338, 4), (17339, 4), (17340, 4), (18957, 4), (18958, 4), (18959, 4), (18960, 4), (18961, 4), (18962, 4), (18963, 4), (18964, 4), (18965, 4), (18966, 4), (530, 3), (531, 3), (532, 3), (533, 3), (534, 3), (535, 3), (536, 3), (537, 3), (538, 3), (539, 3), (9482, 2), (9483, 2), (9484, 2), (9485, 2), (9486, 2), (9487, 2), (94

In [100]:
unique = []
for item in most_common:
    if mapping[item[0]][1] not in unique:
        unique.append(mapping[item[0]][1])

In [101]:
for item in unique[:10]:
    print(item)

data/90s_hiphop/A Tribe Called Quest - Can I Kick It？ (Official HD Video).mp3
data/90s_hiphop/Blackstreet - No Diggity (Official Music Video) ft. Dr. Dre, Queen Pen.mp3
data/90s_hiphop/Total & Keith Murray - Can't You See (Remix) (Official Video).mp3
data/90s_hiphop/Positive K - I Got A Man.mp3
data/country/Kane Brown - Heaven.mp3
data/country/Luke Combs - Beautiful Crazy.mp3
data/country/Toby Keith - As Good As I Once Was.mp3
data/90s_hiphop/The Notorious B.I.G. - Big Poppa (Official Music Video) [HD].mp3
data/2010s_pop/Ed Sheeran - Shape of You (Official Music Video).mp3
data/90s_hiphop/Nas - Nas Is Like (Official Video).mp3


In [75]:
print(len(unique))

29
